In [ ]:
# df.loc[idx[slice(None), '2012-02-15'],]['cs_pctrank_ts_argmax5_closeadj'].unique()  -> for indexing multiindex

In [1]:
import pandas as pd
import nasdaqdatalink as ndl
import json
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer_vect import OpEngineerV
from realgam.quantlib.engineer.ta_engineer_vect import TalibEngineerV
from realgam.quantlib.engineer.alpha_engineer_vect import AlphaEngineerV
from realgam.quantlib.engineer.test_no_class import alpha001, ts_ret
import talib
from joblib import Parallel, delayed
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'
NDL_CONFIG_PATH = f'{PROJECT_PATH}/realgam/quantlib/nasdaq_dl_config.json'
NDL_CONFIG = json.load(open(NDL_CONFIG_PATH))

In [2]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)
stacked_hist = stocks_df.copy()

In [3]:
stacked_hist = stacked_hist.reset_index().set_index(['ticker', 'date'])

In [4]:
stacked_hist[stacked_hist.index.isin(['GGE'], level='ticker')]

open  high   low  close  openadj  highadj  lowadj  \
ticker date                                                            
GGE    2022-03-22  2.90  3.20  2.90   3.02     2.90     3.20    2.90   
       2022-03-23  3.23  3.23  2.86   3.00     3.23     3.23    2.86   

                   closeadj   volume  
ticker date                           
GGE    2022-03-22      3.02  46899.0  
       2022-03-23      3.00  19662.0

In [5]:
tickers_metadata = ndl.get_table('SHARADAR/TICKERS', paginate=True)

In [6]:
# Filtering exchanges and categories of stocks
filter_exchange = NDL_CONFIG['exchanges']

filter_cat = NDL_CONFIG['stock_categories']
# Filter stocks that we focus on
focused_stocks_ticker = tickers_metadata[
    (tickers_metadata.exchange.isin(filter_exchange)) & (
        tickers_metadata.category.isin(filter_cat))][['ticker', 'permaticker']].drop_duplicates()

focused_stocks_ticker['permaticker'] = focused_stocks_ticker['permaticker'].astype('str')

In [7]:
merged = pd.merge(stacked_hist.reset_index(), focused_stocks_ticker, how='left', on=['ticker'])

In [8]:
merged[merged.permaticker.isnull()].ticker.unique()

array(['AFIN', 'ALTM', 'BFRA', 'BOMN', 'BTNB', 'CINR', 'FIG', 'GCAC',
       'GRUB', 'HCCC', 'HFC', 'HGSH', 'IGAP', 'MOTV', 'NXTD', 'NXU',
       'PLIN', 'REPH', 'ROCR', 'SBEA', 'SNII', 'SPAQ', 'TVAC', 'VIAC',
       'VIACA'], dtype=object)

In [11]:
tickers_action = ndl.get_table('SHARADAR/ACTIONS', paginate=True)

In [16]:
tickers_change = tickers_action[tickers_action.action.isin(['tickerchangefrom'])]

In [17]:
tickers_change

,date,action,ticker,name,value,contraticker,contraname
None,,,,,,,
2135,2022-03-24,tickerchangefrom,ACAX,Alset Capital Acquisition Corp,NaN,ACAXU,None
2160,2022-03-23,tickerchangefrom,ZTEK,Zentek Ltd,NaN,ZENYF,None
2357,2022-03-23,tickerchangefrom,GGE,Green Giant Inc,NaN,HGSH,None
2492,2022-03-22,tickerchangefrom,SCTL,Societal Cdmo Inc,NaN,REPH,None
2568,2022-03-22,tickerchangefrom,GHIX,Gores Holdings Ix Inc,NaN,GHIXU,None
...,...,...,...,...,...,...,...
265997,2012-01-05,tickerchangefrom,OPAY,Official Payments Holdings Inc,NaN,TIER,None
266000,2012-01-05,tickerchangefrom,MARK,Remark Holdings Inc,NaN,HSWI,None
266004,2012-01-05,tickerchangefrom,DRD,Drdgold Ltd,NaN,DROOY,None


In [5]:
ae = AlphaEngineerV(small_hist)

In [9]:
ae.alpha7()

ticker  date      
A       2012-04-09    23.0
AAIIQ   2012-04-09    -0.0
AAMRQ   2012-04-09    -0.0
AAON    2012-04-09    51.0
AATC    2012-04-09   -28.0
                      ... 
ABSI    2022-03-21    59.0
ABST    2022-03-21     7.0
ACB     2022-03-21    -9.0
ACEL    2022-03-21    40.0
ACER    2022-03-21   -51.0
Length: 41144, dtype: float64